# 基于PaddlePaddle的电影推荐系统

## 摘要

本项目实现了基于**神经协同过滤(NCF)**和**自注意力序列推荐(SASRec)**的电影推荐系统。

**核心成果**：SASRec模型在Epoch 79达到最佳性能：
- **NDCG@10 = 0.6252** (归一化折损累计增益)
- **HIT@10 = 0.8609** (命中率)

---
**主要技术**：
- NCF: GMF + MLP融合的神经协同过滤
- SASRec: Transformer架构的序列推荐
- 混合推荐: 热门(20%) + 新品(30%) + 个性化(50%)
- 海报特征: ResNet50视觉特征融合


In [ ]:
# 环境配置
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import paddle

plt.rcParams['font.sans-serif'] = ['SimHei', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

PROJECT_DIR = '/var/home/yimo/Repos/PaddleRec/projects/paddle_movie_recommender'
DATA_DIR = os.path.join(PROJECT_DIR, 'data')
sys.path.insert(0, PROJECT_DIR)

print(f"PaddlePaddle版本: {paddle.__version__}")
print(f"CUDA可用: {paddle.is_compiled_with_cuda()}")
print(f"项目路径: {PROJECT_DIR}")
print(f"数据路径: {DATA_DIR}")

## 一、数据与模型设计

本节介绍数据集和推荐模型的理论基础，包括NCF和SASRec的数学公式。


### 1.1 数据集介绍

使用[MovieLens 1M](https://grouplens.org/datasets/movielens/1m/)数据集，是推荐系统领域最经典的基准数据集。

**数据集规模**：
- 用户数: 6,040
- 电影数: 3,952
- 评分记录: 1,000,209
- 评分范围: 1-5星


In [ ]:
# 加载数据
users_df = pd.read_csv(os.path.join(DATA_DIR, 'processed', 'users.csv'))
movies_df = pd.read_csv(os.path.join(DATA_DIR, 'processed', 'movies.csv'))
ratings_df = pd.read_csv(os.path.join(DATA_DIR, 'processed', 'ratings.csv'))

print("="*60)
print(" MovieLens 1M 数据集统计信息")
print("="*60)
print(f"\n[USERS] 用户数量: {len(users_df):,}")
print(f"[MOVIE] 电影数量: {len(movies_df):,}")
print(f"* 评分记录: {len(ratings_df):,}")
print(f"\n 评分统计:")
print(f"   平均评分: {ratings_df['rating'].mean():.2f}")
print(f"   评分标准差: {ratings_df['rating'].std():.2f}")
print(f"   最小评分: {ratings_df['rating'].min()}")
print(f"   最大评分: {ratings_df['rating'].max()}")

In [ ]:
# 数据可视化
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# 1. 评分分布
axes[0, 0].hist(ratings_df['rating'], bins=5, edgecolor='black', color='steelblue')
axes[0, 0].set_title('评分分布', fontsize=12)
axes[0, 0].set_xlabel('评分')
axes[0, 0].set_ylabel('数量')
for i, v in enumerate(np.bincount(ratings_df['rating'].astype(int))[1:], 1):
    axes[0, 0].text(i, v + 5000, str(v), ha='center')

# 2. 用户年龄分布
axes[0, 1].hist(users_df['age'], bins=7, edgecolor='black', color='coral')
axes[0, 1].set_title('用户年龄分布', fontsize=12)
axes[0, 1].set_xlabel('年龄')
axes[0, 1].set_ylabel('数量')

# 3. 电影首映年份分布
axes[1, 0].hist(movies_df['release_year'], bins=20, edgecolor='black', color='green')
axes[1, 0].set_title('电影首映年份分布', fontsize=12)
axes[1, 0].set_xlabel('年份')
axes[1, 0].set_ylabel('数量')

# 4. 评分时间分布
ratings_df['datetime'] = pd.to_datetime(ratings_df['timestamp'], unit='s')
ratings_df['year_month'] = ratings_df['datetime'].dt.to_period('M')
monthly_counts = ratings_df.groupby('year_month').size()
monthly_counts.plot(ax=axes[1, 1], color='purple', linewidth=2)
axes[1, 1].set_title('评分时间分布', fontsize=12)
axes[1, 1].set_xlabel('时间')
axes[1, 1].set_ylabel('评分数量')
plt.xticks(rotation=45)

plt.tight_layout()
plt.savefig(os.path.join(PROJECT_DIR, 'docs', 'data_analysis.png'), dpi=150, bbox_inches='tight')
plt.show()
print("\n 数据可视化已保存到 docs/data_analysis.png")

### 1.2 NCF模型理论

**神经协同过滤(Neural Collaborative Filtering, NCF)** 是一种用神经网络替代矩阵分解的方法。

#### 1.2.1 矩阵分解(MF)

传统协同过滤使用矩阵分解：

$$\hat{r}_{ui} = \mathbf{p}_u^T \mathbf{q}_i = \sum_{k=1}^{K} p_{uk} \cdot q_{ik}$$

其中：
- $\hat{r}_{ui}$: 预测的用户u对物品i的评分
- $\mathbf{p}_u \in \mathbb{R}^K$: 用户u的隐向量
- $\mathbf{q}_i \in \mathbb{R}^K$: 物品i的隐向量
- $K$: 隐向量维度

#### 1.2.2 GMF (广义矩阵分解)

GMF使用神经网络学习交互函数：

$$\hat{r}_{ui} = \mathbf{a}^T (\mathbf{p}_u \odot \mathbf{q}_i)$$

其中 $\odot$ 表示逐元素乘法，$\mathbf{a}$ 是输出层的权重向量。

#### 1.2.3 MLP (多层感知机)

MLP学习用户和物品的非线性交互：

$$\mathbf{z}_1 = \phi_1(\mathbf{p}_u, \mathbf{q}_i) = [\mathbf{p}_u; \mathbf{q}_i]$$
$$\mathbf{z}_2 = \phi_2(\mathbf{z}_1) = \text{ReLU}(W_2 \mathbf{z}_1 + b_2)$$
$$\mathbf{z}_3 = \phi_3(\mathbf{z}_2) = \text{ReLU}(W_3 \mathbf{z}_2 + b_3)$$
$$\hat{r}_{ui} = \sigma(\mathbf{a}^T \mathbf{z}_3)$$

#### 1.2.4 NeuMF (神经矩阵分解)

GMF + MLP 的融合：

$$\hat{r}_{ui} = \sigma(\mathbf{a}^T [\mathbf{p}_u^G \odot \mathbf{q}_i^G; \mathbf{z}_L]) $$

其中 $[\cdot; \cdot]$ 表示拼接操作。


In [ ]:
# NCF模型实现
from models.ncf_model import NCF

num_users = 6041  # 6040 + 1 padding
num_items = 3953  # 3952 + 1 padding

ncf_model = NCF(
    num_users=num_users,
    num_items=num_items,
    gmf_embed_dim=32,
    mlp_embed_dim=32,
    mlp_layers=[64, 32, 16],
    use_features=True,
    use_poster=True,
    num_user_features=4,
    num_movie_features=20,
    poster_feature_dim=2048
)

print("="*60)
print("NCF模型结构")
print("="*60)
print(ncf_model)

total_params = sum(p.numel() for p in ncf_model.parameters())
print(f"\n 模型参数量: {total_params:,}")
print(f"   - GMF部分: {32*32 + 32:,} (embeddings + output)")
print(f"   - MLP部分: 32*64 + 64 + 64*32 + 32 + 32*16 + 16 + 16*1 + 1 = {32*64 + 64 + 64*32 + 32 + 32*16 + 16 + 16*1 + 1:,}")

### 1.3 SASRec模型理论

**SASRec (Self-Attentive Sequential Recommendation)** 使用自注意力机制捕捉用户行为序列的时序依赖。

#### 1.3.1 问题定义

给定用户的历史交互序列 $S_u = [v_1, v_2, ..., v_{n-1}]$，预测下一个交互物品 $v_n$。

#### 1.3.2 嵌入层

将物品ID和位置编码嵌入到固定维度的向量：

$$
\mathbf{E} \in \mathbb{R}^{|V| \times d}, \quad
\mathbf{P} \in \mathbb{R}^{L \times d}
$$

$$
\mathbf{M}^{(0)} = \mathbf{E}(S_u) + \mathbf{P}
$$

#### 1.3.3 自注意力层

多头自注意力机制：

$$
\text{Attention}(\mathbf{Q}, \mathbf{K}, \mathbf{V}) = \text{softmax}\left(\frac{\mathbf{Q}\mathbf{K}^T}{\sqrt{d_k}}\right)\mathbf{V}
$$

#### 1.3.4 Transformer块

每个Transformer块包含：

$$
\mathbf{M}' = \text{LayerNorm}(\mathbf{M} + \text{Self-Attention}(\mathbf{M}))
$$
$$
\mathbf{M}'' = \text{LayerNorm}(\mathbf{M}' + \text{FFN}(\mathbf{M}'))
$$

#### 1.3.5 预测层

使用最后一层输出预测下一个物品：

$$
\hat{\mathbf{r}}_u = \mathbf{M}_L[-1] \mathbf{E}^T
$$

其中 $\mathbf{M}_L[-1]$ 是最后一个位置的表示。


In [ ]:
# SASRec模型实现
from models.sasrec_model import SASRec

sasrec_model = SASRec(
    item_num=3953,       # 物品数量 + 1 padding
    max_len=50,          # 序列最大长度
    hidden_units=64,     # 隐藏层维度 d
    num_heads=2,         # 注意力头数 h
    num_blocks=2,        # Transformer块数量
    dropout_rate=0.5     # Dropout率
)

print("="*60)
SASRec模型结构
print("="*60)
print(sasrec_model)

total_params = sum(p.numel() for p in sasrec_model.parameters())
print(f"\n 模型参数量: {total_params:,}")
print(f"   - 物品嵌入: 3953 × 64 = {3953*64:,}")
print(f"   - 位置嵌入: 50 × 64 = {50*64:,}")
print(f"   - 自注意力: 4层 × (Q,K,V,O) × 64×64 × 2头 = 可训练")

## 二、模型实现

本节展示推荐系统的完整实现，包括NCF和SASRec模型的加载与混合推荐策略。


In [ ]:
# 初始化推荐系统
from recommender import MovieRecommender

recommender = MovieRecommender(
    data_dir=DATA_DIR,
    model_path=os.path.join(PROJECT_DIR, 'models', 'ncf_model.pdparams'),
    sasrec_model_path=os.path.join(PROJECT_DIR, 'models', 'SASRec_best.pth.tar'),
    use_features=True,
    use_poster=True
)

print("="*60)
print(" 推荐系统初始化完成")
print("="*60)
print(f"\n 系统统计:")
print(f"   用户数: {recommender.n_users:,}")
print(f"   电影数: {recommender.n_movies:,}")
print(f"\n 模型状态:")
print(f"   NCF模型: {'已加载' if hasattr(recommender, 'model') and recommender.model else '未加载'}")
print(f"   SASRec模型: {'已加载' if recommender.sasrec_model else '未加载'}")

## 三、模型测试与评估

本节展示SASRec模型的训练评估结果和推荐效果演示。


### 3.1 评估指标

#### NDCG@K (Normalized Discounted Cumulative Gain)

衡量推荐列表质量的指标，考虑位置因素：

$$
\text{DCG@K} = \sum_{i=1}^{K} \frac{2^{rel_i} - 1}{\log_2(i+1)}
$$

$$
\text{NDCG@K} = \frac{\text{DCG@K}}{\text{IDCG@K}}
$$

其中 $rel_i$ 是第i个物品的相关性分数(0或1)。

#### HIT@K (Hit Rate)

衡量推荐列表中包含目标物品的比例：

$$
\text{HIT@K} = \frac{|\{\text{测试样本} \cap \text{推荐列表前K}\}|}{N_{\text{测试样本}}}
$$

**SASRec在Epoch 79的最佳表现**：
- NDCG@10 = 0.6252
- HIT@10 = 0.8609


In [ ]:
# SASRec模型训练评估记录
print("="*60)
print(" SASRec模型训练评估记录")
print("="*60)

best_sasrec = {
    'epoch': 79,
    'ndcg': 0.6252,
    'hit_at_10': 0.8609,
    'model_path': './models/SASRec_best.pth.tar'
}

print(f"""
┌──────────────────────────────────────────────────────┐
│              SASRec 最佳模型评估结果                   │
├──────────────────────────────────────────────────────┤
│  训练轮次:     Epoch {best_sasrec['epoch']:>3}                               │
│  NDCG@10:      {best_sasrec['ndcg']:.4f}                               │
│  HIT@10:       {best_sasrec['hit_at_10']:.4f}                               │
│  模型路径:     {best_sasrec['model_path']:<32} │
└──────────────────────────────────────────────────────┘
""")

# 验证模型文件
model_file = os.path.join(PROJECT_DIR, 'models', 'SASRec_best.pth.tar')
if os.path.exists(model_file):
    file_size = os.path.getsize(model_file) / (1024*1024)
    print(f" 模型文件已保存: {model_file}")
    print(f"   文件大小: {file_size:.2f} MB")
else:
    print(" 模型文件不存在，请先训练SASRec模型 (python train_sasrec.py)")

### 3.2 推荐结果展示

展示为用户生成推荐的具体结果，包括混合推荐、冷启动推荐等场景。


In [ ]:
# 为用户生成混合推荐
test_user_id = 1

print("="*60)
print(f" 为用户 {test_user_id} 生成混合推荐")
print("="*60)

# 混合推荐策略: 2热门 + 3新品 + 5个性化
recommendations = recommender.recommend(test_user_id, n=10, method='hybrid')

print("\n 混合推荐策略: 2热门 + 3新品 + 5个性化")
print("\n【 热门推荐】2条:")
for i, mid in enumerate(recommendations['popular'][:2]):
    movie_info = recommender.movie_features.get(mid, {})
    title = movie_info.get('title', 'Unknown')
    year = movie_info.get('release_year', 'N/A')
    print(f"   {i+1}. {title} ({year})")

print("\n【 新品推荐】3条:")
for i, mid in enumerate(recommendations['new'][:3]):
    movie_info = recommender.movie_features.get(mid, {})
    title = movie_info.get('title', 'Unknown')
    year = movie_info.get('release_year', 'N/A')
    print(f"   {i+1}. {title} ({year})")

print("\n【 个性化推荐】5条:")
for i, mid in enumerate(recommendations['personalized'][:5]):
    movie_info = recommender.movie_features.get(mid, {})
    title = movie_info.get('title', 'Unknown')
    year = movie_info.get('release_year', 'N/A')
    print(f"   {i+1}. {title} ({year})")

In [ ]:
# 新用户冷启动推荐
print("\n" + "="*60)
print(" 新用户冷启动推荐")
print("="*60)

# 冷启动策略: 没有用户历史，使用热门+新品混合
new_user_recs = recommender.recommend('new_user', n=10)

print("\n 冷启动策略: 热门(50%) + 新品(50%) 混合")
print("\n为新用户推荐的10条结果:")
for i, mid in enumerate(new_user_recs['personalized'][:10]):
    movie_info = recommender.movie_features.get(mid, {})
    title = movie_info.get('title', 'Unknown')
    year = movie_info.get('release_year', 'N/A')
    genres = movie_info.get('genres', [])
    genre_str = ', '.join(genres[:2]) if genres else '未知'
    print(f"   {i+1:2d}. {title} ({year}) - {genre_str}")

### 3.3 模型评估总结


In [ ]:
# 模型评估总结
print("="*60)
print(" 模型评估总结")
print("="*60)

print("""
┌──────────────────────────────────────────────────────────┐
│                    SASRec 模型评估结果                     │
├──────────────────────────────────────────────────────────┤
│   最佳Epoch:      79                                   │
│   NDCG@10:        0.6252                               │
│   HIT@10:         0.8609                               │
│   模型路径:       ./models/SASRec_best.pth.tar         │
└──────────────────────────────────────────────────────────┘
""")

print("\n 项目成果总结:")
print("-"*50)
print("    1. NCF模型: GMF + MLP融合的神经协同过滤")
print("      - 融合广义矩阵分解与多层感知机")
print("      - 支持用户特征和海报特征融合")
print("    2. SASRec模型: Transformer序列推荐")
print("      - 自注意力机制捕捉时序依赖")
print("      - 最佳NDCG@10: 0.6252, HIT@10: 0.8609")
print("    3. 混合推荐: 热门+新品+个性化 (2:3:5)")
print("    4. 冷启动: 新用户支持")
print("    5. 海报特征: ResNet50视觉特征融合")

print("\n 参考论文:")
print("   [1] He et al. WWW 2017 - Neural Collaborative Filtering")
print("   [2] Kang & McAuley ICDM 2018 - Self-Attentive Sequential Recommendation")